# Momepy

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-04-18T21:06:33.928712-04:00

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import momepy as mm  # requires main
import networkx as nx

from core import utils

%watermark -w
%watermark -iv

Watermark: 2.4.3

networkx : 3.3
momepy   : 0.7.1.dev29+g593fc8e
geopandas: 0.14.3



In [3]:
city = "Auckland"
utils.city_fua[city]

869

In [4]:
auckland = utils.read_original(utils.city_fua[city])

Simplification using `roundabout_simplification` method to tackle roundabouts.

In [5]:
simple = mm.roundabout_simplification(auckland)

/Users/jgaboardi/miniforge3/envs/py311_uscuni_simplification/lib/python3.11/site-packages/momepy/preprocessing.py:765: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '24.20261626845682' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  rab_adj.loc[g.Index, "hdist"] = hdist


The function has quite a few parameters and fairly smart logic of determining the _main road_ using COINS algorithm.

In [6]:
mm.roundabout_simplification?

Signature:
mm.roundabout_simplification(
    edges,
    polys=None,
    area_col='area',
    circom_threshold=0.7,
    area_threshold=0.85,
    include_adjacent=True,
    diameter_factor=1.5,
    center_type='centroid',
    angle_threshold=0,
)
Docstring:
Selects the roundabouts from ``polys`` to create a center point to merge all
incoming edges. If None is passed, the function will perform shapely polygonization.

All ``edges`` attributes are preserved and roundabouts are deleted.
Note that some attributes, like length, may no longer reflect the reality of newly
constructed geometry.

If ``include_adjacent`` is True, adjacent polygons to the actual roundabout are
also selected for simplification if two conditions are met:
    - the area of adjacent polygons is less than the actual roundabout
    - adjacent polygons do not extend beyond a factor of the diameter of the actual
    roundabout.
    This uses hausdorff_distance algorithm.

Parameters
----------
edges : GeoDataFrame
    GeoD

The other option is to use `consolidate_intersections` which works more based on graph properties.

In [7]:
graph = mm.gdf_to_nx(auckland, length="length", integer_labels=True)

We need to add additional edge attributes _from_ and _to_ at the moment. Seems redundant, will need to refactor.

In [8]:
nx.set_edge_attributes(
    graph, {edge: {"from": edge[0], "to": edge[1]} for edge in graph.edges}
)

In [9]:
simple_graph = mm.consolidate_intersections(graph)

It does not have a ton of relevant options but shall work better in preserving planarity I think.